# Correr experimentos

In [15]:
import math, subprocess
import numpy as np
import os
from IPython.display import display, clear_output

In [16]:
def correr_experimento(algoritmo, secuencia1, secuencia2):
    # Crear proceso para ejecutar el codigo.
    process = subprocess.Popen(["../../cli", "-a", algoritmo, "-s", secuencia1, "-t", secuencia2], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)
    
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    assert exit_code == 0, F"Hubo un error en la experimentacion para el algoritmo: {algoritmo} con la secuencia {secuencia1} y {secuencia2}."
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    outputs = process.stderr.readline().split(); 
    
    tiempo = outputs[0]
    celdas = outputs[1]
    score = outputs[2]
    longitud1 = outputs[3]
    longitud2 = outputs[4]
    
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();

    return tiempo, celdas, score, longitud1, longitud2

In [17]:
experimentos = [];

# Experimento NW

In [18]:
genomes = [f"../../sequences/genomes/{name}" for name in os.listdir("../../sequences/genomes/")]

for genome1 in genomes:
    for genome2 in genomes:
        if genome1 == genome2:
            continue
        for lenguaje in ["C","ASM"]:
            for tecnologia in ["LIN","SSE","AVX","AVX512"]:
                algoritmo = f"NW_{lenguaje}_{tecnologia}"
                experimentos.append([algoritmo, genome1, genome2])

randoms = [f"../../sequences/random/{name}" for name in os.listdir("../../sequences/random/")]

for random1 in randoms:
    random1_len = random1.split('_')[1]
    for random2 in randoms:
        random2_len = random2.split('_')[1]
        if random1_len != random2_len or random1 == random2:
            continue
        for lenguaje in ["C","ASM"]:
            for tecnologia in ["LIN","SSE","AVX","AVX512"]:
                algoritmo = f"NW_{lenguaje}_{tecnologia}"
                experimentos.append([algoritmo, random1, random2])

# Experimento SW

In [13]:
ref_seq_dirs = [name for name in os.listdir("../../sequences/reads/")]

ref_seq_and_read_pairs = []

for ref_dir in ref_seq_dirs:
    ref_seq = f"../../sequences/genomes/{ref_dir}.fasta"
    for name in os.listdir(f"../../sequences/reads/{ref_dir}"):
        ref_seq_read = f"../../sequences/reads/{ref_dir}/{name}" 
        ref_seq_and_read_pairs.append((ref_seq, ref_seq_read))

for seq_pairs in ref_seq_and_read_pairs:
    for lenguaje in ["C","ASM"]:
        for tecnologia in ["LIN","SSE","AVX","AVX512"]:
            algoritmo = f"SW_{lenguaje}_{tecnologia}"
            experimentos.append([algoritmo, seq_pairs[0], seq_pairs[1]])

In [14]:
columnas = [
    "algoritmo",
    "secuencia1",
    "secuencia2",
    "longitud1",
    "longitud2",
    "tiempo",
    "celdas",
    "score"
];

filas = [];
numero = 1
T = 1 # Numero de veces que se ejecuta cada experimento
for experimento in experimentos:
    
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    algoritmo = experimento[0]
    secuencia1 = experimento[1]
    secuencia2 = experimento[2]
    
    tiempos = []
      
    for i in range(0, T):
        tiempo, celdas, score, longitud1, longitud2 = correr_experimento(algoritmo, secuencia1, secuencia2)
        tiempos.append(tiempo);
        
    tiempo_promedio = np.median(tiempos);
    
    filas.append([
        algoritmo,
        secuencia1,
        secuencia2,
        longitud1,
        longitud2,
        tiempo_promedio,
        celdas,
        score]
    );


In [ ]:
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("../results/resultado.csv", index=False, header=True);